# 1) Titolo e obiettivi

Lezione 30: Testo e Dato - Trasformare testo in feature numeriche

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, perché vettorizzare il testo | 5 min |
| 2 | Teoria: preprocessing, BoW, n-grammi, sparsità | 15 min |
| 3 | Schema mentale: workflow text → vector | 5 min |
| 4 | Demo: preprocessing, CountVectorizer, parametri | 25 min |
| 5 | Esercizi guidati + classificazione NB | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | **Pulire e tokenizzare** il testo | Sai rimuovere punteggiatura e lowercase? |
| 2 | Costruire **Bag of Words** con CountVectorizer | Sai creare matrice sparsa da documenti? |
| 3 | Usare **n-grammi** per catturare contesto | Sai settare ngram_range? |
| 4 | Gestire **sparsità** e vocabolario | Sai usare max_features, min_df? |
| 5 | **Trasformare nuovi testi** correttamente | Sai usare transform() senza refit? |

---

## L'idea centrale: dal testo ai numeri

```
TESTO GREZZO:                       DOPO PREPROCESSING:          BOW MATRIX:
                                                                  the  cat  sat  dog  ran
"The cat sat."                      "the cat sat"                [  1    1    1    0    0  ]
"The dog ran!"                      "the dog ran"                [  1    0    0    1    1  ]

          │                               │                              │
     Pulizia                        Tokenizzazione                 Vettorizzazione
   (lowercase,                      (split su spazi)              (conteggio parole)
    rimuovi !, .)
```

**Perché serve:** i modelli ML vogliono numeri, non stringhe!

---

## Il problema della sparsità

```
Vocabolario: 10.000 parole
Documento medio: 50 parole

Matrice BoW:
┌─────────────────────────────────────────────────────┐
│ 0  0  0  1  0  0  0  2  0  0  0  0  1  0  0  0  ... │  ← 99.5% zeri!
│ 0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  ... │
└─────────────────────────────────────────────────────┘

Sparsità = (n_zeri / n_totali) * 100
```

**Soluzione:** `max_features`, `min_df`, rappresentazioni sparse in scipy.

---

## N-grammi: catturare contesto

| N-gram | Esempio | Pro | Contro |
|--------|---------|-----|--------|
| Unigram (1) | "not", "good" | Semplice | Perde "not good" |
| Bigram (2) | "not good" | Cattura negazione | 2x vocabolario |
| Trigram (3) | "not very good" | Più contesto | 3x vocabolario |

```python
# Solo unigrammi:
CountVectorizer(ngram_range=(1, 1))

# Uni + bigrammi:
CountVectorizer(ngram_range=(1, 2))
```

---

## Workflow NLP base

```
┌──────────┐    ┌──────────┐    ┌──────────┐    ┌──────────┐
│  Testo   │ →  │  Pulizia │ →  │  Fit     │ →  │ Matrice  │
│  grezzo  │    │ lowercase│    │ Vectorizer│   │  sparsa  │
└──────────┘    │ punct    │    └──────────┘    └──────────┘
                └──────────┘           │
                                       ▼
                              Nuovo testo? → transform() SOLO!
```

**Regola d'oro:** fit() su train, transform() su train e test.

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| Stringhe Python | Base Python | Sai usare lower(), split()? |
| Regex base | re module | Sai usare re.sub()? |
| Matrici sparse | scipy | Sai che esistono formati CSR? |
| Classificazione | Lezioni 5-6 | Sai cos'è MultinomialNB? |

**Cosa useremo:** CountVectorizer, TfidfVectorizer (preview), MultinomialNB, regex per pulizia.

# 2) Teoria concettuale
- Il testo va normalizzato (lowercase, rimozione punteggiatura/numeri) prima di essere vettorizzato.
- Bag of Words: rappresentazione vettoriale basata su conteggi; input = lista di documenti, output = matrice sparsa (n_doc x vocab_size).
- N-grammi: catturano contesto locale combinando parole consecutive (es. bigrammi). Pi? n-grammi aumentano dimensione e sparsimita'.
- Sparsimita': la maggior parte delle celle e' zero; serve per capire memoria e performance.


## CountVectorizer vs TF-IDF (preview)
- CountVectorizer conta occorrenze; sensibile a parole frequenti ma non informative.
- TF-IDF ripesa le parole in base alla rarita' nel corpus; utile per classificazione testo.
- Entrambi richiedono vocabolario fisso: fit sul training, transform sul test/nuovi testi.


# 3) Schema mentale / mappa decisionale
1. Pulizia minima (lowercase, punteggiatura, numeri, spazi multipli).
2. Tokenizzazione semplice o tramite CountVectorizer.
3. Scelta n-grammi (1,1 per bag of words base; (1,2) per includere bigrammi).
4. Valuta sparsimita' e dimensione vocabolario; opziona max_features/min_df per controllarla.
5. Trasforma nuovi documenti solo con `transform`, non rifittare il vocabolario.


# 4) Sezione dimostrativa
- Demo 1: preprocessing e tokenizzazione manuale.
- Demo 2: Bag of Words con CountVectorizer, sparsimita' e n-grammi.
- Demo 3: parametri chiave (max_features, min_df) e trasformazione di nuovi testi.


## Demo 1 - Preprocessing e tokenizzazione
Perche': capire cosa succede prima del vectorizer e controllare gli output attesi.


## Demo 2 - Bag of Words e sparsimita'
Perche': vedere vocabolario, matrice sparsa e impatto degli n-grammi.


## Demo 3 - Parametri e nuovi documenti
Perche': controllare la dimensione del vocabolario e applicare trasformazioni coerenti a nuovi testi.


In [ ]:
# Setup NLP e corpus di esempio
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

# Corpus di recensioni (ASCII)
corpus = [
    "Ottimo prodotto, qualita eccellente!",
    "Prodotto scadente, non lo consiglio",
    "Qualita buona, prezzo giusto",
    "Non funziona, prodotto difettoso",
    "Consiglio questo prodotto, ottimo acquisto",
    "Prezzo alto ma qualita ottima",
    "Prodotto arrivato rotto, pessima esperienza",
    "Acquisto consigliato, spedizione veloce"
]
print(f"Numero documenti: {len(corpus)}")
assert len(corpus) > 0


In [ ]:
# Demo 1: preprocessing manuale
# Scopo: mostrare lowercase + rimozione punteggiatura/numeri/spazi multipli.

def preprocess_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # punteggiatura -> spazio
    text = re.sub(r'\d+', ' ', text)       # numeri -> spazio
    text = re.sub(r'\s+', ' ', text).strip()
    return text

cleaned = [preprocess_text(doc) for doc in corpus]
print(cleaned[:3])
assert all(isinstance(t, str) for t in cleaned)


In [ ]:
# Demo 1 (continua): tokenizzazione semplice

def tokenize(text: str) -> list:
    return text.split()

for i, doc in enumerate(cleaned[:3], 1):
    tokens = tokenize(doc)
    print(f"Doc {i} tokens: {tokens}")


In [ ]:
# Demo 2: Bag of Words con CountVectorizer
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus)
print(f"Shape BoW: {X_bow.shape}")
print("Vocabolario:", list(vectorizer.get_feature_names_out()))
assert X_bow.shape[0] == len(corpus)


In [ ]:
# Demo 2 (continua): analisi della sparsimita'
n_total = X_bow.shape[0] * X_bow.shape[1]
n_nonzero = X_bow.nnz
sparsity = 1 - (n_nonzero / n_total)
print(f"Elementi totali: {n_total}, non-zero: {n_nonzero}, sparsity: {sparsity:.1%}")
assert 0 <= sparsity <= 1


In [ ]:
# Demo 2 (continua): n-grammi (unigram + bigram)
vec_bigram = CountVectorizer(ngram_range=(1,2))
X_bigram = vec_bigram.fit_transform(corpus)
vocab_uni = vectorizer.get_feature_names_out()
vocab_bi = vec_bigram.get_feature_names_out()
print(f"Vocabolario unigram: {len(vocab_uni)} parole")
print(f"Vocabolario uni+bigram: {len(vocab_bi)} features")
assert X_bigram.shape[0] == len(corpus)


In [ ]:
# Demo 3: parametri CountVectorizer
print("Max features e min_df per controllare il vocabolario")
vec_limited = CountVectorizer(max_features=10)
X_lim = vec_limited.fit_transform(corpus)
print(f"Vocabolario max_features=10: {list(vec_limited.get_feature_names_out())}")

vec_min_df = CountVectorizer(min_df=2)
vec_min_df.fit(corpus)
print(f"Vocabolario min_df=2: {list(vec_min_df.get_feature_names_out())}")


In [ ]:
# Demo 3 (continua): trasformare nuovi documenti
nuovi = ["Prodotto eccellente e prezzo onesto", "Spedizione lenta e prodotto difettoso"]
X_new = vectorizer.transform(nuovi)
print(f"Shape nuovi documenti: {X_new.shape}")
print("Prima riga (densa)", X_new[0].toarray())
assert X_new.shape[1] == X_bow.shape[1]


In [ ]:
# ============================================================
# TRASFORMAZIONE DI NUOVI DOCUMENTI
# ============================================================
# Importante: usare .transform() (non fit_transform) per nuovi dati

print("=" * 60)
print("TRASFORMARE NUOVI DOCUMENTI")
print("=" * 60)

# Vectorizer già addestrato (su corpus)
print("\nVectorizer addestrato sul corpus originale")
print(f"Vocabolario: {list(vectorizer.get_feature_names_out())}")

# Nuovo documento da trasformare
nuovo_doc = ["Prodotto eccellente, prezzo ottimo"]
print(f"\nNuovo documento: \"{nuovo_doc[0]}\"")

# CORRETTO: usare transform (non fit_transform)
X_nuovo = vectorizer.transform(nuovo_doc)

print(f"\nVettore risultante:")
print(f"  Shape: {X_nuovo.shape}")
print(f"  Valori: {X_nuovo.toarray()[0]}")

# Quali parole sono state riconosciute?
parole_riconosciute = []
for i, count in enumerate(X_nuovo.toarray()[0]):
    if count > 0:
        parole_riconosciute.append(vectorizer.get_feature_names_out()[i])

print(f"\nParole riconosciute dal vocabolario: {parole_riconosciute}")

# Parole nuove (out-of-vocabulary) vengono ignorate
print("\n⚠️  NOTA: 'eccellente' non era nel vocabolario → IGNORATA")
print("    Questo è un limite del BoW: parole nuove scompaiono")

# 5) Esercizi svolti (passo-passo)
## Esercizio 30.1 - Pipeline di preprocessing
Obiettivo: applicare pulizia, tokenizzazione e rimozione stopword italiane, restituendo liste di token.


In [ ]:
# Soluzione esercizio 30.1
STOPWORDS_IT = {'il','lo','la','i','gli','le','un','una','di','a','da','in','con','su','per','tra','fra','e','o','ma','che','non','sono','ha','ho','questo','questa','questi','queste','come','quando','dove','perche','chi','cosa','piu','molto','anche','solo','ancora','se','ma'}

def preprocess_and_tokenize(texts):
    tokens_list = []
    for t in texts:
        clean = preprocess_text(t)
        tokens = [tok for tok in tokenize(clean) if tok not in STOPWORDS_IT]
        tokens_list.append(tokens)
    return tokens_list

processed = preprocess_and_tokenize(corpus)
print(processed)
assert len(processed) == len(corpus)


## Esercizio 30.2 - BoW manuale
Obiettivo: costruire manualmente matrice BoW e confrontarla con CountVectorizer.


In [ ]:
# Soluzione esercizio 30.2

def manual_bag_of_words(texts):
    cleaned = [preprocess_text(t) for t in texts]
    tokens = [tokenize(t) for t in cleaned]
    vocab = sorted({tok for doc in tokens for tok in doc})
    vocab_index = {w:i for i,w in enumerate(vocab)}
    bow = np.zeros((len(tokens), len(vocab)), dtype=int)
    for i, doc in enumerate(tokens):
        for tok in doc:
            bow[i, vocab_index[tok]] += 1
    return bow, vocab

bow_manual, vocab_manual = manual_bag_of_words(corpus)
print(f"Shape manuale: {bow_manual.shape}, vocab size: {len(vocab_manual)}")
assert bow_manual.shape[0] == len(corpus)


## Esercizio 30.3 - Sentiment con BoW
Obiettivo: classificare recensioni positive/negative usando MultinomialNB su BoW.


In [ ]:
# Soluzione esercizio 30.3
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

recensioni = [
    ("Prodotto eccezionale, lo consiglio a tutti", 1),
    ("Ottimo acquisto, qualita eccellente", 1),
    ("Pessimo, arrivato rotto", 0),
    ("Non funziona, soldi sprecati", 0),
    ("Prezzo onesto e buona qualita", 1),
    ("Esperienza terribile, non comprare", 0),
]
texts, labels = zip(*recensioni)
vec = CountVectorizer()
X = vec.fit_transform(texts)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42, stratify=labels)

nb = MultinomialNB()
nb.fit(X_train, y_train)
preds = nb.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Accuracy: {acc:.3f}")
print(classification_report(y_test, preds, digits=3))
assert acc > 0


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **Pulisci PRIMA di vettorizzare** | Lowercase + rimuovi punteggiatura |
| 2 | **fit() su train, transform() su tutto** | Vocabolario consistente |
| 3 | **Sparsità è normale** | 95%+ zeri è tipico, usa matrici sparse |
| 4 | **N-grammi = più contesto, più dimensioni** | Trade-off da gestire |
| 5 | **max_features/min_df per controllare** | Limita vocabolario troppo grande |

---

## Confronto sintetico: parametri CountVectorizer

| Parametro | Default | Effetto | Quando cambiare |
|-----------|---------|---------|-----------------|
| `ngram_range` | (1,1) | Solo unigrammi | (1,2) per bigrammi |
| `max_features` | None | Tutti i termini | 5000-10000 per limitare |
| `min_df` | 1 | Anche termini rari | 2-5 per rimuovere rari |
| `max_df` | 1.0 | Anche termini comuni | 0.9 per rimuovere troppo comuni |
| `stop_words` | None | Nessuna rimozione | 'english' o lista custom |

---

## Perché questi concetti funzionano

### 1) Bag of Words come conteggio

```
Documento: "the cat sat on the mat"

Vocabolario: {cat: 0, mat: 1, on: 2, sat: 3, the: 4}

Vettore BoW: [1, 1, 1, 1, 2]
              ↑  ↑  ↑  ↑  ↑
             cat mat on sat the (2 volte!)
```

### 2) Perché fit() e transform() separati

```
TRAINING:                          TEST:
fit_transform(train_docs)          transform(test_docs)
     ↓                                   ↓
Impara vocabolario              Usa STESSO vocabolario
+ trasforma                     Parole nuove → ignorate
```

Se rifitti su test, il vocabolario cambia → disastro!

---

## Reference card metodi

| Metodo | Input | Output | Note |
|--------|-------|--------|------|
| `CountVectorizer()` | - | vectorizer object | Configurazione |
| `.fit(docs)` | lista di stringhe | self | Impara vocabolario |
| `.transform(docs)` | lista di stringhe | sparse matrix | Applica vocabolario |
| `.fit_transform(docs)` | lista di stringhe | sparse matrix | fit + transform |
| `.get_feature_names_out()` | - | array di termini | Vocabolario |
| `.toarray()` | sparse matrix | dense array | Attenzione memoria! |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| Refit su test | Vocabolario diverso | Solo transform() |
| Dimenticare lowercase | "The" ≠ "the" | lowercase=True o preprocessing |
| Vocabolario enorme | Lento, sparse | max_features, min_df |
| Stopwords sbagliate | Lingua diversa | Specifica lingua o lista |
| toarray() su grande | Memoria esplode | Tieni sparse |

---

## Template preprocessing + BoW

```python
import re
from sklearn.feature_extraction.text import CountVectorizer

# 1) Funzione pulizia
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # solo lettere e spazi
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 2) Applica pulizia
docs_clean = [preprocess(doc) for doc in docs_raw]

# 3) Vettorizza
vec = CountVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
X_train = vec.fit_transform(train_docs_clean)
X_test = vec.transform(test_docs_clean)

# 4) Verifica
print(f"Vocabolario: {len(vec.get_feature_names_out())} termini")
print(f"Sparsità: {100 * (1 - X_train.nnz / (X_train.shape[0] * X_train.shape[1])):.1f}%")
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 31 | TF-IDF | Pesatura per importanza |
| 32 | Sentiment Analysis | Classificazione di polarità |
| 33+ | NER, Document Intelligence | Entità e struttura |

# 7) Checklist di fine lezione
- [ ] Ho pulito e tokenizzato il testo prima della vettorizzazione.
- [ ] Ho controllato la dimensione del vocabolario e la sparsimita'.
- [ ] Ho scelto n-grammi e parametri (max_features/min_df) adeguati.
- [ ] Ho trasformato i nuovi testi con lo stesso vectorizer fit.
- [ ] Ho testato una pipeline di classificazione (es. NB) con BoW.

Glossario
- Tokenizzazione: suddivisione del testo in parole.
- Bag of Words: rappresentazione a conteggi di parole.
- N-gramma: sequenza di n parole consecutive.
- Sparsimita': percentuale di zeri in una matrice.
- Vocabolario: insieme di termini mappati a colonne.
- Stopword: parole molto frequenti e poco informative.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-02-01 | Creazione: preprocessing e BoW base |
| 1.1 | 2024-02-08 | Aggiunto n-grammi e sparsità |
| 2.0 | 2024-02-15 | Integrata classificazione NB |
| 2.1 | 2024-02-20 | Refactor con checkpoint |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII text→vector workflow, sparsità diagram, n-grammi comparison, 5 take-home messages, tabella parametri CountVectorizer, fit/transform separation, template completo preprocessing + BoW, reference card metodi |

---

## Note per lo studente

Questa lezione apre il **modulo NLP**:

| Concetto | Lezione | Status |
|----------|---------|--------|
| Testo come dato | 30 (questa) | In corso |
| TF-IDF | 31 | Prossima |
| Sentiment Analysis | 32 | Dopo |
| NER | 33 | Dopo |
| Document Intelligence | 34 | Dopo |

**Pipeline NLP base:**
1. Pulizia testo (regex, lowercase)
2. Tokenizzazione (split o vectorizer)
3. Vettorizzazione (BoW, TF-IDF)
4. Modello ML (NB, LogReg, ...)

**Prossima tappa:** Lesson 31 - TF-IDF per pesare le parole importanti